# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
output_data_file = "data_file/cities.csv"
city_data = pd.read_csv(output_data_file)

city_data_df = pd.DataFrame(city_data)
city_data_df.sample(20)


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
239,chekhov,90,RU,1609831708,92,55.14,37.47,30.20,6.71
74,clyde river,1,CA,1609831695,67,70.47,-68.59,-22.00,6.93
310,codrington,100,AU,1609831713,81,-38.27,141.97,57.85,14.67
49,west bay,0,KY,1609831693,73,19.37,-81.42,77.00,1.01
143,coquimbo,22,CL,1609831700,87,-29.95,-71.34,59.00,2.24
114,andenes,75,NO,1609831697,93,69.31,16.12,39.99,16.11
325,lumphat,89,KH,1609831714,42,13.49,106.98,90.32,7.27
420,gat,0,IL,1609831642,100,31.61,34.76,61.00,3.47
321,barabai,100,ID,1609831714,62,-2.58,115.38,84.70,1.95
451,stornoway,75,GB,1609831723,86,58.21,-6.39,36.00,3.36


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key.
gmaps.configure(api_key=g_key)

# Store 'Latitude' and 'Longitude' into  locations. 
locations = city_data_df[["Lat", "Lng"]].astype(float)

# Create a humidity Heatmap layer
humidity = city_data_df["Humidity"]
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(humidity),
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Narrow down the DataFrame to find your ideal weather condition.
ideal_conditions_df = city_data_df

# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 15 mph.
# Zero cloudiness.
ideal_conditions_df= ideal_conditions_df.loc[(ideal_conditions_df["Max Temp"] < 80) & (ideal_conditions_df["Max Temp"] > 70)]
ideal_conditions_df = ideal_conditions_df.loc[ideal_conditions_df["Wind Speed"] < 15]
ideal_conditions_df = ideal_conditions_df.loc[ideal_conditions_df["Cloudiness"] == 0]

# Drop any rows with null values
ideal_conditions_df = ideal_conditions_df.dropna()

ideal_conditions_df.sample(20)

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
356,bilma,0,NE,1609831716,18,18.69,12.92,75.88,7.18
483,birao,0,CF,1609831725,22,10.28,22.79,77.29,10.89
297,along,0,IN,1609831712,27,28.17,94.77,77.41,1.28
465,maketu,0,NZ,1609831724,82,-37.77,176.45,71.01,1.99
131,sur,0,OM,1609831698,31,22.57,59.53,72.46,12.77
178,save,0,BJ,1609831702,85,8.04,2.49,76.80,3.13
163,porbandar,0,IN,1609831504,46,21.64,69.61,73.40,11.41
503,mecca,0,SA,1609831522,66,21.43,39.83,76.64,1.39
234,abu dhabi,0,AE,1609831595,33,24.47,54.37,71.60,4.70
185,sokoto,0,NG,1609831703,21,13.06,5.24,72.93,9.40


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = ideal_conditions_df

hotel_df['Hotel Name'] = ''

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}


for index, hotel in hotel_df.iterrows():
    Lat = hotel["Lat"]
    Lng = hotel["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = (f"{Lat},{Lng}")
    
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    response = requests.get(url, params=params).json()
    
    # extract results
    closest_hotel = response['results']
    
    try:
        print(f"Results for {index}: {hotel['City']}.")
        print(f"Closest hotel is {closest_hotel[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = closest_hotel[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Results for 49: west bay.
Closest hotel is Calypso Cove.
------------
Results for 95: ponedera.
Closest hotel is HOTEL SHALOM.
------------
Results for 123: salalah.
Closest hotel is HAMDAN PLAZA HOTEL.
------------
Results for 131: sur.
Closest hotel is Sur Plaza Hotel.
------------
Results for 163: porbandar.
Closest hotel is Lords Inn Porbandar.
------------
Results for 178: save.
Closest hotel is Mission Ahmadiyya Savè.
------------
Results for 185: sokoto.
Closest hotel is RG Palace.
------------
Results for 220: lazaro cardenas.
Closest hotel is Collection O Hotel Quinta Antigua.
------------
Results for 234: abu dhabi.
Closest hotel is Beach Rotana.
------------
Results for 259: suleja.
Closest hotel is Capy Motel.
------------
Results for 297: along.
Closest hotel is Hotel West.
------------
Results for 306: umm lajj.
Closest hotel is Moon light Furnished Units.
------------
Results for 314: cockburn town.
Closest hotel is Osprey Beach Hotel.
------------
Results for 356: bilma

In [8]:
#Updated with 'Hotel Name' column
hotel_df.sample(15)

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
185,sokoto,0,NG,1609831703,21,13.06,5.24,72.93,9.40,RG Palace
459,casa nova,0,BR,1609831723,73,-9.42,-41.13,75.20,14.99,Pousada e Restaurante Dona Xinha
465,maketu,0,NZ,1609831724,82,-37.77,176.45,71.01,1.99,Maketu Beach Holiday Park
163,porbandar,0,IN,1609831504,46,21.64,69.61,73.40,11.41,Lords Inn Porbandar
95,ponedera,0,CO,1609831696,65,10.64,-74.75,78.80,6.93,HOTEL SHALOM
306,umm lajj,0,SA,1609831712,54,25.02,37.27,75.81,6.67,Moon light Furnished Units
356,bilma,0,NE,1609831716,18,18.69,12.92,75.88,7.18,Expeditions Ténére voyages
483,birao,0,CF,1609831725,22,10.28,22.79,77.29,10.89,
386,bealanana,0,MG,1609831718,52,-14.55,48.73,78.46,3.33,
178,save,0,BJ,1609831702,85,8.04,2.49,76.80,3.13,Mission Ahmadiyya Savè


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**hotel) for index, hotel in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)

#Pins
hotel_pins = gmaps.symbol_layer(
    locations, scale=2,
    info_box_content=hotel_info)

# Display figure
fig.add_layer(markers)
fig.add_layer(hotel_pins)
fig

Figure(layout=FigureLayout(height='420px'))